# Basics of Decision Making Course

**- Pedram Peiro Asfia 9825008**

**- Mahdi Mohammadi 9825041**

**- Mohadese Najafzadeh 9825502**

# Libraries

In [1]:
import numpy as np
import pandas as pd
from numpy import linalg
import copy
from scipy.stats import rankdata

# Input

These rankings will not reach the consensus level.

In [2]:
r1 = (2,3,4,5,1)
r2 = (3,5,4,1,2)
r3 = (2,3,4,5,1)
r4 = (2,3,5,4,1)
r5 = (1,3,5,2,4)
r6 = (2,3,4,5,1)
r7 = (3,4,2,5,1)
r8 = (4,3,5,1,2)
r9 = (1,3,4,5,2)
r10 = (2,4,5,3,1)
r11 = (4,2,3,5,1)
r12 = (2,3,4,5,1)
r13 = (3,2,4,5,1)
r14 = (1,5,3,4,2)
r15 = (5,1,3,4,2)
r16 = (2,5,4,3,1)
r17 = (4,3,5,2,1)
r18 = (3,1,2,5,4)
r19 = (1,5,3,2,4)
r20 = (5,1,2,3,4)

# number of alternatives
num_alternatives = 5

# number of experts
m = 20

# number of clusters
num_clusters = 3

# threshold of acceptance
lambd = 0.75

These rankings will reach the consensus level.

In [3]:
# r1 = (3,2,4,5,1)
# r2 = (3,4,5,1,2)
# r3 = (3,2,4,5,1)
# r4 = (3,2,5,4,1)
# r5 = (1,3,5,2,4)
# r6 = (3,2,4,5,1)
# r7 = (2,3,4,5,1)
# r8 = (3,4,5,2,1)
# r9 = (2,3,4,5,1)
# r10 = (2,3,5,4,1)
# r11 = (3,2,4,5,1)
# r12 = (3,2,4,5,1)
# r13 = (3,2,4,5,1)
# r14 = (2,3,5,4,1)
# r15 = (4,1,3,5,2)
# r16 = (3,2,4,5,1)
# r17 = (4,2,3,5,1)
# r18 = (3,2,1,5,4)
# r19 = (2,4,5,3,1)
# r20 = (4,1,3,5,2)

# # number of alternatives
# num_alternatives = 5

# # number of experts
# m = 20

# # number of clusters
# num_clusters = 3

# # threshold of acceptance
# lambd = 0.75

In [4]:
rankings = {'r'+str(i): np.array(eval(f'r{i}')) for i in range(1,21)}
for x in list(rankings.keys()):
    print(f'{x}: {rankings[x]}')

r1: [2 3 4 5 1]
r2: [3 5 4 1 2]
r3: [2 3 4 5 1]
r4: [2 3 5 4 1]
r5: [1 3 5 2 4]
r6: [2 3 4 5 1]
r7: [3 4 2 5 1]
r8: [4 3 5 1 2]
r9: [1 3 4 5 2]
r10: [2 4 5 3 1]
r11: [4 2 3 5 1]
r12: [2 3 4 5 1]
r13: [3 2 4 5 1]
r14: [1 5 3 4 2]
r15: [5 1 3 4 2]
r16: [2 5 4 3 1]
r17: [4 3 5 2 1]
r18: [3 1 2 5 4]
r19: [1 5 3 2 4]
r20: [5 1 2 3 4]


# Function

First cluster centroid

In [5]:
def first_initializer(rankings):
    '''
    Desc:
        calculate the distance of each score given by experts to others'
        
    input:
        o rankings: dictionary, rankings given by the experts
    
    output: 
        o tuple, name of the expert and its score
    
    '''
    num = len(list(rankings.values())[0])
    av_dist = dict()
    for x in list(rankings.keys()):
        score_x = rankings[x]
        dist = np.mean([linalg.norm(score_x - rankings[y] , 2) for y in 
                        list(rankings.keys()) if y!=x])
        av_dist[x] = dist
    
    # ordering based on values
    av_dist = dict( sorted(av_dist.items(), key=lambda item: item[1], reverse=False) )
    key = list(av_dist.keys())[0]
    val = rankings[key]
    
    return (key , val)

Other clustering centroids

In [6]:
def others_initializers(rankings , centroids):
    '''
    Desc: 
        calculate the average distance of other points to the previously selected centroids 
        in order to select a new centroid
        
    input:
        o rankings: dictionary, rankings given by the experts
        o centroids: list of previously selected centroids
        
    output:
        o tuple, name of the expert and its score
    '''
    
    ranks = copy.deepcopy(rankings)
    for cent in centroids: 
        del ranks[cent[0]]
    
    av_dist = dict()
    for x in list(ranks.keys()):
        score_x = ranks[x]
        dist = np.mean([linalg.norm(score_x - cent[1] , 2) for cent in centroids])
        av_dist[x] = dist
        
    # ordering based on values
    av_dist = dict( sorted(av_dist.items(), key=lambda item: item[1] , reverse=True) )
    key = list(av_dist.keys())[0]
    val = rankings[key]
    
    return (key,val)

Improved min-max Initialization of clustering centroids

In [7]:
def improved_min_max_initialization(rankings , k):
    '''
    Desc:
        random initialization of cluster's centroids with improved max-min method proposed in the article
    
    input:
        o rankings: dictionary, rankings gievn by the experts
        o k: int, number of clusters
        
    output:
        o k initialization points
    
    '''
    
    centroids = []
    
    first_centroid = first_initializer(rankings)
    centroids.append(first_centroid)
    
    for i in range(k-1):
        others = others_initializers(rankings , centroids)
        centroids.append(others)
    
    L = len(centroids)
    centroids = {'C'+str(i): centroids[i-1] for i in range(1,L+1)}
    return centroids

Euclidean distance between 2 points (based on the article)

In [8]:
def euclidean_distance(point1 , point2):
    '''
    Desc:
         euclidean distance calculator between 2 points based on article
    
    input:
        o point1: np.array
        o point2: np.array
        
    output:
        o distance between 2 points
    '''
    n = len(point1)
    d = np.sum((point1-point2)**2)
    
    denom = 2*np.sum([(n-i)**2 for i in range(1 , n , 2)])
    
    dist = np.sqrt(d/denom)
    return dist

Assigning cluster labels to each expert's ranking 

In [9]:
def assigning_to_nearest_clusters(rankings , centroids):
    '''
    Desc:
         assigning rankings of experts to each cluster's centroid
    
    input:
        o rankings: dictionary, rankings gievn by the experts
        o centroids: list of tuples, (name , value)
        
    output:
        o labels of each expert ranking
    '''
    labels = []
    for rank in rankings.values():
        dist = []
        for cent in centroids.values():
            dist.append(euclidean_distance(rank , cent[1]))
        labels.append('C'+str(np.argmin(dist)+1))
    return labels

# Testing

Calculate the preference vector of the rankings inside a cluster

In [10]:
def preference_vector_cluster(rankings , num_alternatives):
    '''
    Desc:
         calculate the preference vector of a number of rankings
    
    input:
        o rankings: np.array, rankings gievn by the experts
        o num_alternatives: int, numeber of alternatives
        
    output:
        o preference vector of a set of rankings
    '''
    
    denom = np.sum(np.arange(num_alternatives))
    
    w = (num_alternatives-rankings)/denom
    w_C = np.mean(w , axis = 0)

    return w_C

Identification rule for clusters (Phase I)

In [11]:
def identification_rule_clusters(SOCI, lambd):
    '''
    Desc:
         It is used to identify the cluster Cl that does not reach the predefined threshold γ
    
    input:
        o SOCI: dict, cluster's names with their subgroup ordinal consensus index
        o lambd: float, threshold of acceptance
        
    output:
        o identification_cluster: np.array, working as a mask to identify the clusters that SOCI<lambda
    '''
    identification_cluster = np.array(list(SOCI.values()))<lambd
    
    return identification_cluster

Identification rule for alternatives (Phase II)

In [12]:
def identification_rule_alternatives(chosen_clusters , C_G):
    '''
    Desc:
         It is used to identify the alternatives that should be modified by Cl
    
    input:
        o chosen_clusters: dict, cluster's names with their ordinal prefernces 
        
    output:
        o AL: alternatives that should be modified by Cl
    '''
    AL = dict()

    for x in list(chosen_clusters.keys()):    
        diff = np.abs(chosen_clusters[x] - C_G)
        alters = np.argwhere(diff == np.amax(diff))
        AL[x] = np.array(alters.squeeze().reshape(-1,)+1)
    return AL

Identification rule for pairs of alternatives

In [13]:
def Identification_rule_pairs_of_alternatives(num_alternatives , AL ,chosen_clusters , C_G ):
    '''
    Desc:
          this rule identifies pairwise alternatives (xi, xj) whose mutual preference 
          relationships farthest with the global group’s
    
    input:
        o num_alternatives: int, number of alternatives
        o AL: dict, alternatives that should be modified by Cl
        o cluster_names: list, name of all of the clusters
        o C_G: ordinal preferences of all the experts
        
    output:
        o alternatives_pairs: dict, pair of alternatives that should be modified
        o direction_rule: dict, (r^(Cl)_i − r^(Cl)_j) − (r^G_i − r^G_j)
    '''
        
    alternatives_pairs=dict()
    direction_rule=dict()

    for c in list(AL.keys()):
        place_holder = 0
        for i in (AL[c]-1):
            for j in range(num_alternatives):
                diff = np.abs(chosen_clusters[c][i]-chosen_clusters[c][j] - (C_G[i]-C_G[j]))
                if diff>place_holder:
                    alternatives_pairs[c] = (i+1,j+1)
                    place_holder=diff
        else:
            (i,j) = alternatives_pairs[c]
            temp = chosen_clusters[c][i-1]-chosen_clusters[c][j-1] - (C_G[i-1]-C_G[j-1])
            direction_rule[c] = temp
    return {'alternatives pairs':alternatives_pairs,
           'direction rule':direction_rule}

# Process

In [14]:
def LSGDM_decision_making(rankings ,num_alternatives , m , num_clusters , lambd):
    centroids = improved_min_max_initialization(rankings , num_clusters)

    print(f'Centroids of the clusters are as follows:')
    for x in list(centroids.keys()):
        print(f'Cluster {x}: {centroids[x]}')


    rankings_clusters = np.array(assigning_to_nearest_clusters(rankings ,centroids))
    print('\nThe cluster assinged to each expert\'s ranking:')
    for i in range(m):
        print(f'r{i+1} is in cluster {rankings_clusters[i]}')


    C_l_preference = dict()
    size = dict()
    for i in range(num_clusters):
        mask = rankings_clusters=='C'+str(i+1)
        x = preference_vector_cluster(np.array(list(rankings.values()))[mask] ,num_alternatives )
        C_l_preference['C'+str(i+1)] = x
        size['C'+str(i+1)] = np.sum(mask)/m

    print('\nPreference vector for clusters and their size are:')
    for x in list(C_l_preference.keys()):
        print(f'Cluster {x}\'s prefernce vector: {C_l_preference[x]}')
        print(f'Cluster {x}\'s size: {size[x]*m}\n')

    C_l = dict()
    print('Ordinal preferences of subgroups:')
    for x in list(C_l_preference.keys()):
        C_l[x] = np.abs(rankdata(C_l_preference[x] , method='ordinal') - (num_alternatives+1))
        print(f'Ordinal preference of subgroup {x}: {C_l[x]}')

    C_G_preference = preference_vector_cluster(np.array(list(rankings.values())) , num_alternatives)
    C_G = np.abs(rankdata(C_G_preference, method = 'ordinal') - (num_alternatives+1))
    print(f'\nPreference vector of the group {C_G_preference}')
    print(f'Ordinal preference of group {C_G}\n')


    SOCI = dict()
    GOCI=0
    for x in C_l.keys():
        SOCI[x] = (1-euclidean_distance(C_l[x] , C_G))
        print(f'SOCI for {x} is {SOCI[x]}')
        GOCI+=size[x]*SOCI[x]
    else: print('GOCI is' , GOCI,'\n\n')
    
    if GOCI>lambd:
        t = {'x'+str(i+1): C_G[i] for i in range(num_alternatives)}
        t = dict( sorted(t.items(), key=lambda item: item[1] , reverse=False) )
        t = list(t.keys())
        print('\033[1m\033[92mFinal ranking of alternatives:','>'.join(t)+'\033[0m')
    else:
        print('\n\033[1mIdentification Rules:\033[0m')
        print('\033[93mPhase I\033[0m')
        identification_cluster = identification_rule_clusters(SOCI, lambd)
        cluster_names = np.array(['C'+str(i+1) for i in range(num_clusters)])
        print('Clusters that should make modifications are:' , cluster_names[identification_cluster],'\n')

        chosen_clusters = {x: C_l[x] for x in cluster_names[identification_cluster]}
        
        AL = identification_rule_alternatives(chosen_clusters , C_G)
        print('\033[93mPhase II\033[0m')
        for x in AL.keys():
            print(f'The alternatives that should be modified in cluster {x}: {AL[x]}')
        else: print('\n')

        print('\033[93mPhase III\033[0m')    
        res = Identification_rule_pairs_of_alternatives(num_alternatives , AL ,chosen_clusters , C_G )
        alternatives_pairs = res['alternatives pairs']
        direction_rule = res['direction rule']
        C_l_revised=dict()
        i=0
        for c in list(direction_rule.keys()):

            if i>0: print('\n')
            i+=1

            if direction_rule[c]<0:
                print(f'''DMs in subgroup {c} should increase the assessment associated with pair of alternatives (x{alternatives_pairs[c][0]},x{alternatives_pairs[c][1]})''')

                C_l_revised[c] = copy.deepcopy(C_l[c])

                C_l_revised[c][alternatives_pairs[c][0]-1] = C_l[c][alternatives_pairs[c][1]-1]
                C_l_revised[c][alternatives_pairs[c][1]-1] = C_l[c][alternatives_pairs[c][0]-1]

                print(f'Previous ordinal preferences of subgroup {c}: {C_l[c]}')
                print(f'Suggested ordinal preferences of subgroup {c}: {C_l_revised[c]}')
            else:
                print(f'''DMs in subgroup {c} should decrease the assessment associated with pair of alternatives (x{alternatives_pairs[c][0]},x{alternatives_pairs[c][1]})''')

                C_l_revised[c] = copy.deepcopy(C_l[c])
                
                C_l_revised[c][alternatives_pairs[c][0]-1] = C_l[c][alternatives_pairs[c][1]-1]
                C_l_revised[c][alternatives_pairs[c][1]-1] = C_l[c][alternatives_pairs[c][0]-1]


                print(f'Previous ordinal preferences of subgroup {c}: {C_l[c]}')
                print(f'Suggested ordinal preferences of subgroup {c}: {C_l_revised[c]}')
        else:
            print('\n\033[1m\033[31mAfter adjusting the rankings, run this function another time\033[0m')

# Output

In [15]:
LSGDM_decision_making(rankings ,num_alternatives , m , num_clusters , lambd)

Centroids of the clusters are as follows:
Cluster C1: ('r1', array([2, 3, 4, 5, 1]))
Cluster C2: ('r20', array([5, 1, 2, 3, 4]))
Cluster C3: ('r19', array([1, 5, 3, 2, 4]))

The cluster assinged to each expert's ranking:
r1 is in cluster C1
r2 is in cluster C3
r3 is in cluster C1
r4 is in cluster C1
r5 is in cluster C3
r6 is in cluster C1
r7 is in cluster C1
r8 is in cluster C1
r9 is in cluster C1
r10 is in cluster C1
r11 is in cluster C1
r12 is in cluster C1
r13 is in cluster C1
r14 is in cluster C1
r15 is in cluster C2
r16 is in cluster C1
r17 is in cluster C1
r18 is in cluster C2
r19 is in cluster C3
r20 is in cluster C2

Preference vector for clusters and their size are:
Cluster C1's prefernce vector: [0.25714286 0.17142857 0.1        0.09285714 0.37857143]
Cluster C1's size: 14.0

Cluster C2's prefernce vector: [0.06666667 0.4        0.26666667 0.1        0.16666667]
Cluster C2's size: 3.0

Cluster C3's prefernce vector: [0.33333333 0.06666667 0.1        0.33333333 0.16666667]
Clu